# **Hàm xử lý cột judgement**

In [4]:
# -*- coding: utf-8 -*-
# Các thư viện cần thiết
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
import json # Để xử lý cột 'judgement'

# --- Bước 0: Định nghĩa hàm parse_judgement_revised (từ các thảo luận trước) ---
def parse_judgement_revised(judgement_str):
    # Giá trị mặc định
    avg_time_ms = np.nan
    avg_memory_kb = np.nan
    num_failed_overall_tc = 0 
    num_accepted_tc = 0
    total_verdicts_tc = 0 
    num_tc_from_array = 0 
    
    num_wrong_tc = 0
    num_tle_tc = 0    
    num_mle_tc = 0    
    num_rte_tc = 0    
    num_ce_tc = 0     
    is_compile_error = False 

    default_series = pd.Series({
        'avg_time_ms': np.nan, 'avg_memory_kb': np.nan,
        'num_failed_overall_tc': np.nan, 'num_accepted_tc': np.nan,
        'total_verdicts_tc': np.nan, 'num_tc_from_array': np.nan,
        'num_wrong_tc': np.nan, 'num_tle_tc': np.nan, 
        'num_mle_tc': np.nan, 'num_rte_tc': np.nan, 'num_ce_tc': np.nan,
        'is_compile_error': False 
    })

    if pd.isna(judgement_str) or not isinstance(judgement_str, str) or judgement_str.strip() == "":
        return default_series

    try:
        data = json.loads(judgement_str) 
        times_list = data.get("times", [])
        mems_list = data.get("mems", [])
        verdicts_data = data.get("verdicts", {})

        num_tc_from_array = len(times_list)

        valid_times = [t for t in times_list if isinstance(t, (int, float)) and t > 0]
        if valid_times:
            avg_time_ms = sum(valid_times) / len(valid_times)

        valid_mems = [m for m in mems_list if isinstance(m, (int, float)) and m > 0]
        if valid_mems:
            avg_memory_kb = sum(valid_mems) / len(valid_mems)

        if isinstance(verdicts_data, dict):
            for verdict_key, count in verdicts_data.items():
                v_upper = str(verdict_key).upper() # Đảm bảo key là string trước khi upper
                # Kiểm tra lỗi biên dịch
                # (có thể cần danh sách từ khóa đầy đủ hơn hoặc dùng regex)
                if "ERROR:" in str(verdict_key) or \
                   "DECLARED IN THIS SCOPE" in v_upper or \
                   "DOES NOT NAME A TYPE" in v_upper or \
                   "SYNTAX ERROR" in v_upper: # Thêm Syntax Error vào check CE
                    is_compile_error = True
                    current_ce_count = count if isinstance(count, int) and count > 0 else 1
                    num_ce_tc += current_ce_count
                    # Coi lỗi biên dịch là một dạng lỗi chính
                    num_failed_overall_tc += current_ce_count 
                elif isinstance(count, int) and count > 0: # Các verdict chuẩn
                    total_verdicts_tc += count
                    if v_upper == "WRONG" or v_upper == "WA":
                        num_wrong_tc += count
                        num_failed_overall_tc += count
                    elif v_upper == "ACCEPTED" or v_upper == "AC":
                        num_accepted_tc += count
                    elif v_upper in ["TIME LIMIT EXCEEDED", "TLE"]:
                        num_tle_tc += count
                        num_failed_overall_tc += count
                    elif v_upper in ["MEMORY LIMIT EXCEEDED", "MLE"]:
                        num_mle_tc += count
                        num_failed_overall_tc += count
                    elif v_upper in ["RUNTIME ERROR", "RTE"]:
                        num_rte_tc += count
                        num_failed_overall_tc += count
        elif isinstance(verdicts_data, list) and not verdicts_data: 
            pass # List rỗng, không có verdict nào để xử lý
            
        if is_compile_error and num_ce_tc > 0:
             # Nếu có lỗi biên dịch, các kết quả test case khác có thể không còn ý nghĩa
             # hoặc không được thực thi. Reset chúng để phản ánh điều này.
             num_wrong_tc = 0; num_accepted_tc = 0; num_tle_tc = 0
             num_mle_tc = 0; num_rte_tc = 0
             # total_verdicts_tc có thể được coi là số lỗi CE, hoặc 0 nếu không có test nào chạy
             total_verdicts_tc = num_ce_tc if num_ce_tc > 0 else 0 
             num_failed_overall_tc = num_ce_tc # Tổng lỗi chính là lỗi CE

    except (json.JSONDecodeError, TypeError, Exception) as e:
        # print(f"Error parsing judgement: {judgement_str} -> {e}") # Bỏ comment để debug nếu cần
        return default_series

    return pd.Series({
        'avg_time_ms': avg_time_ms, 'avg_memory_kb': avg_memory_kb,
        'num_failed_overall_tc': num_failed_overall_tc, 'num_accepted_tc': num_accepted_tc,
        'total_verdicts_tc': total_verdicts_tc, 'num_tc_from_array': num_tc_from_array,
        'num_wrong_tc': num_wrong_tc, 'num_tle_tc': num_tle_tc,
        'num_mle_tc': num_mle_tc, 'num_rte_tc': num_rte_tc, 'num_ce_tc': num_ce_tc,
        'is_compile_error': is_compile_error
    })


# **Xử lý dữ liệu**

In [31]:
print("--- A. Bắt đầu xử lý annonimized.csv ---")

# 1. Tải Dữ liệu
annonimized_file_path = '/kaggle/input/dataset-csv/annonimized.csv' 
try:
    df_submissions = pd.read_csv(annonimized_file_path)
    print(f"\n1. Tải thành công '{annonimized_file_path}'. Số dòng: {len(df_submissions)}, số cột: {len(df_submissions.columns)}")
    print("   Tên các cột gốc:", df_submissions.columns.tolist())
except FileNotFoundError:
    print(f"LỖI: File '{annonimized_file_path}' không tìm thấy. Vui lòng kiểm tra lại đường dẫn.")
    exit()
except Exception as e:
    print(f"LỖI khi tải '{annonimized_file_path}': {e}")
    exit()

column_rename_map = {
    "concat('it001',`assignment_id`)" : 'assignment_id_raw',
    "concat('it001',`problem_id`)" : 'problem_id_raw',
    "concat('it001', username)" : 'username_raw',
    "concat('it001',`language_id`)" : 'language_id_raw',
}


actual_renames = {old: new for old, new in column_rename_map.items() if old in df_submissions.columns}
df_submissions.rename(columns=actual_renames, inplace=True)
print("\n2. Tên cột sau khi đổi:", df_submissions.columns.tolist())

df_submissions.head(5)

--- A. Bắt đầu xử lý annonimized.csv ---

1. Tải thành công '/kaggle/input/dataset-csv/annonimized.csv'. Số dòng: 295198, số cột: 11
   Tên các cột gốc: ["concat('it001',`assignment_id`)", "concat('it001',`problem_id`)", "concat('it001', username)", 'is_final', 'status', 'pre_score', 'coefficient', "concat('it001',`language_id`)", 'created_at', 'updated_at', 'judgement']

2. Tên cột sau khi đổi: ['assignment_id_raw', 'problem_id_raw', 'username_raw', 'is_final', 'status', 'pre_score', 'coefficient', 'language_id_raw', 'created_at', 'updated_at', 'judgement']


,assignment_id_raw,problem_id_raw,username_raw,is_final,status,pre_score,coefficient,language_id_raw,created_at,updated_at,judgement
0,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,10-09 08:02:04,10-09 08:06:58,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."
1,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,10-09 08:04:41,10-09 08:04:51,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."
2,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 08:06:49,10-09 08:06:58,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."
3,90ce27571176d87961b565d5ef4b3de33ede04ac,bf96fbdc5f499538c3e2bfbec5779c8a14b0a9ff,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 08:47:52,10-09 08:48:01,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."
4,90ce27571176d87961b565d5ef4b3de33ede04ac,7a6e5ca470ff47c3b5048f240c4738de71010c78,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 09:19:35,10-09 09:19:45,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."


In [28]:
# Áp dụng hàm parse_judgement_revised cho cột 'judgement'
judgement_features_df = df_submissions['judgement'].apply(parse_judgement_revised)


# Đếm số lần xuất hiện trong từng cột
for column in judgement_features_df.columns:
    print(f"Số lần xuất hiện của các giá trị trong cột '{column}':")
    print(judgement_features_df[column].value_counts())
    print("\n" + "-"*50 + "\n")


Số lần xuất hiện của các giá trị trong cột 'avg_time_ms':
avg_time_ms
0.020000    1434
0.520000    1354
0.530000    1243
0.510000    1153
0.540000     727
            ... 
0.079474       1
0.073889       1
0.072632       1
0.073684       1
0.729333       1
Name: count, Length: 7744, dtype: int64

--------------------------------------------------

Số lần xuất hiện của các giá trị trong cột 'avg_memory_kb':
avg_memory_kb
1984.000000     21419
10044.000000      772
2128.000000       561
2140.000000       545
1988.000000       487
                ...  
11257.866667        1
11276.533333        1
24706.100000        1
24364.900000        1
2794.736842         1
Name: count, Length: 4868, dtype: int64

--------------------------------------------------

Số lần xuất hiện của các giá trị trong cột 'num_failed_overall_tc':
num_failed_overall_tc
0.0      129713
1.0       42420
10.0      29630
2.0       13258
3.0       10207
          ...  
78.0          1
96.0          1
121.0         1
118.0  

In [30]:
# Ta nên loại bỏ các cột chỉ có một loại giá trị, hoặc phân bố không đều

judgement_features_df.drop(columns = ['num_rte_tc','num_mle_tc','num_accepted_tc'])

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,avg_time_ms,avg_memory_kb,num_failed_overall_tc,total_verdicts_tc,num_tc_from_array,num_wrong_tc,num_tle_tc,num_ce_tc,is_compile_error
0,NaN,NaN,10.0,10.0,10.0,10.0,0.0,0.0,False
1,NaN,NaN,10.0,10.0,10.0,10.0,0.0,0.0,False
2,NaN,NaN,0.0,0.0,10.0,0.0,0.0,0.0,False
3,NaN,NaN,0.0,0.0,10.0,0.0,0.0,0.0,False
4,NaN,NaN,0.0,0.0,10.0,0.0,0.0,0.0,False
...,...,...,...,...,...,...,...,...,...
295193,NaN,NaN,0.0,0.0,14.0,0.0,0.0,0.0,False
295194,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,True
295195,NaN,NaN,0.0,0.0,14.0,0.0,0.0,0.0,False
295196,NaN,NaN,0.0,0.0,13.0,0.0,0.0,0.0,False


In [32]:
# 3. Trích xuất Thông tin Định danh
if 'username_raw' in df_submissions.columns:
    df_submissions['student_id'] = df_submissions['username_raw'].astype(str).str.replace("it001", "", regex=False)
    print("   Đã tạo cột 'student_id'.")
else:
    print("LƯU Ý: Cột 'username_raw' không tìm thấy. Bỏ qua tạo 'student_id'.")


if 'language_id_raw' in df_submissions.columns:
    df_submissions['language_id'] = df_submissions['language_id_raw'].astype(str).str.replace("it001", "", regex=False)
    print("   Đã tạo cột 'language_id'.")
else:
    print("LƯU Ý: Cột 'language_id_raw' không tìm thấy. Bỏ qua tạo 'language_id'.")

# Giữ lại assignment_id_raw và problem_id_raw (có thể xử lý thêm nếu cần sau này)
if 'assignment_id_raw' not in df_submissions.columns and "concat('it001',`assignment_id`)" in actual_renames.values():
     print(f"Cột assignment_id_raw không được tạo dù có trong map, kiểm tra tên gốc: { {k:v for k,v in column_rename_map.items() if v == 'assignment_id_raw'} }")


print("\n3. Các cột sau khi trích xuất ID (một phần):", df_submissions.columns.tolist())


placeholder_year = 2025
if 'created_at' in df_submissions.columns:
    df_submissions['created_at'] = pd.to_datetime(df_submissions['created_at'], format='%d-%m %H:%M:%S', errors='coerce')
    df_submissions['created_at'] = df_submissions['created_at'].apply(lambda dt: dt.replace(year=placeholder_year) if pd.notnull(dt) else dt)
    print(f"   'created_at' chuyển sang datetime (gán năm {placeholder_year}).")

if 'updated_at' in df_submissions.columns:
    df_submissions['updated_at'] = pd.to_datetime(df_submissions['updated_at'], format='%d-%m %H:%M:%S', errors='coerce')
    df_submissions['updated_at'] = df_submissions['updated_at'].apply(lambda dt: dt.replace(year=placeholder_year) if pd.notnull(dt) else dt)
    print(f"   'updated_at' chuyển sang datetime (gán năm {placeholder_year}).")


# 5. Phân tích (Parse) Cột `judgement`
if 'judgement' in df_submissions.columns:
    df_submissions.drop(columns = ['judgement'])
    print("\n5. Đã xóa cột 'judgement'...")

#Nối 2 bảng
df_submissions = pd.concat([df_submissions.reset_index(drop=True), judgement_features_df.reset_index(drop=True)], axis=1)
print("\n. Đã nối các cột parse'...")

# 6. Xử lý Cột `status` (thông tin thêm)
if 'status' in df_submissions.columns:
    print("\n6. Phân tích cột 'status':")
    print("   Các giá trị duy nhất trong cột 'status':", df_submissions['status'].unique().tolist())
    # Kiểm tra các submission 'pending'
    pending_submissions = df_submissions[df_submissions['status'] == 'pending']
    print(f"   Số lượng submission có status 'pending': {len(pending_submissions)}")
    if not pending_submissions.empty:
        pass
else:
    print("LƯU Ý: Cột 'status' không tìm thấy.")


print("\n7. Kiểm tra dữ liệu thiếu (NaN) trong df_submissions đã xử lý:")
nan_counts_submissions = df_submissions.isnull().sum()
print(nan_counts_submissions[nan_counts_submissions > 0]) # Chỉ in những cột có NaN

df_submissions_processed = df_submissions.copy()
print("\n   Hoàn thành tiền xử lý cho df_submissions.")
print("   Thông tin df_submissions_processed:")
df_submissions_processed.info()
print("\n   5 dòng đầu của df_submissions_processed:")
df_submissions_processed.head()

   Đã tạo cột 'student_id'.
   Đã tạo cột 'language_id'.

3. Các cột sau khi trích xuất ID (một phần): ['assignment_id_raw', 'problem_id_raw', 'username_raw', 'is_final', 'status', 'pre_score', 'coefficient', 'language_id_raw', 'created_at', 'updated_at', 'judgement', 'student_id', 'language_id']
   'created_at' chuyển sang datetime (gán năm 2025).
   'updated_at' chuyển sang datetime (gán năm 2025).

5. Đã xóa cột 'judgement'...

5. Đã nối các cột parse'...

6. Phân tích cột 'status':
   Các giá trị duy nhất trong cột 'status': ['SCORE', 'Compilation Error', 'pending', 'Syntax Error']
   Số lượng submission có status 'pending': 10

7. Kiểm tra dữ liệu thiếu (NaN) trong df_submissions đã xử lý:
created_at               165375
updated_at               165601
avg_time_ms              266848
avg_memory_kb            259895
num_failed_overall_tc        10
num_accepted_tc              10
total_verdicts_tc            10
num_tc_from_array            10
num_wrong_tc                 10
num_tle_

,assignment_id_raw,problem_id_raw,username_raw,is_final,status,pre_score,coefficient,language_id_raw,created_at,updated_at,...,num_failed_overall_tc,num_accepted_tc,total_verdicts_tc,num_tc_from_array,num_wrong_tc,num_tle_tc,num_mle_tc,num_rte_tc,num_ce_tc,is_compile_error
0,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,2025-09-10 08:02:04,2025-09-10 08:06:58,...,10.0,0.0,10.0,10.0,10.0,0.0,0.0,0.0,0.0,False
1,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,2025-09-10 08:04:41,2025-09-10 08:04:51,...,10.0,0.0,10.0,10.0,10.0,0.0,0.0,0.0,0.0,False
2,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2025-09-10 08:06:49,2025-09-10 08:06:58,...,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,False
3,90ce27571176d87961b565d5ef4b3de33ede04ac,bf96fbdc5f499538c3e2bfbec5779c8a14b0a9ff,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2025-09-10 08:47:52,2025-09-10 08:48:01,...,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,False
4,90ce27571176d87961b565d5ef4b3de33ede04ac,7a6e5ca470ff47c3b5048f240c4738de71010c78,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,2025-09-10 09:19:35,2025-09-10 09:19:45,...,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,False


In [33]:
# 1. Sau khi tiền xử lý df_submissions
print("\n   Đang lưu dữ liệu đã xử lý vào 'annonimized_processed.csv'...")

# Lưu lại df_submissions_processed vào file CSV
output_file_path = 'annonimized_processed.csv'
df_submissions_processed.to_csv(output_file_path, index=False)

print(f"   Dữ liệu đã được lưu thành công vào '{output_file_path}'.")


   Đang lưu dữ liệu đã xử lý vào 'annonimized_processed.csv'...
   Dữ liệu đã được lưu thành công vào 'annonimized_processed.csv'.
